<a href="https://colab.research.google.com/github/Lucas01iveira/data_science-alura/blob/master/Curso-ModelosDeClassifica%C3%A7%C3%A3o/Introducao_ModelosDeClassificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Definindo a Spark Session

In [ ]:
!pip install PySpark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.9 MB/s eta 0:00:00


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
      .master('local[*]')\
      .appName('Classificação com Spark')\
      .getOrCreate()

spark

In [ ]:
# aproveito para já criar uma conexão com o repositório do drive
from google.colab import drive 
drive.mount('/content/my-drive')

Mounted at /content/my-drive


## Carregamento dos dados

In [ ]:
dados = spark.read.csv(
    path='/content/my-drive/MyDrive/Colab_Notebooks/Alura-CursoSparkModelosClassificacao/*.csv',
    sep=',',
    header=True,
    inferSchema=True 
)

In [ ]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



In [ ]:
dados.dtypes

[('id', 'int'),
 ('Churn', 'string'),
 ('Mais65anos', 'int'),
 ('Conjuge', 'string'),
 ('Dependentes', 'string'),
 ('MesesDeContrato', 'int'),
 ('TelefoneFixo', 'string'),
 ('MaisDeUmaLinhaTelefonica', 'string'),
 ('Internet', 'string'),
 ('SegurancaOnline', 'string'),
 ('BackupOnline', 'string'),
 ('SeguroDispositivo', 'string'),
 ('SuporteTecnico', 'string'),
 ('TVaCabo', 'string'),
 ('StreamingFilmes', 'string'),
 ('TipoContrato', 'string'),
 ('ContaCorreio', 'string'),
 ('MetodoPagamento', 'string'),
 ('MesesCobrados', 'double')]

In [ ]:
dados.show(truncate=False)

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|id |Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|Internet   |SegurancaOnline   |BackupOnline      |SeguroDispositivo |SuporteTecnico    |TVaCabo           |StreamingFilmes   |TipoContrato|ContaCorreio|MetodoPagamento |MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|0  |Nao  |0         |Sim    |Nao        |1              |Nao         |SemServicoTelefonico    |DSL        |Nao               |Sim               |Nao           

In [ ]:
# será que a distribuição de pessoas que estão fazendo churn (cancelando o serviço) é equilibrada?
dados.groupBy('Churn').count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|  Sim| 5174|
|  Nao| 5174|
+-----+-----+



## Transformando os dados (pt1)

- Ao efetuar um printSchema() pode-se notar que a maioria das colunas (variáveis/features) são do tipo string. Entretanto isso é um problema para os modelos de Machine Learning, que precisam de números concretos para funcionar.
(a máquina não consegue incluir registros tipo string de maneira consistente no modelo)

- É necessário pensar numa maneira de contornar esse problema

- Essa seção se dedica à apresentação de códigos pensados por mim

In [ ]:
dados.show(10,truncate=False)

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|id |Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|Internet   |SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio|MetodoPagamento |MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+
|0  |Nao  |0         |Sim    |Nao        |1              |Nao         |SemServicoTelefonico    |DSL        |Nao            |Sim         |Nao              |Nao           |Nao    |Nao            |Mensalmente |Sim         |BoletoEletronico|29.85  

In [ ]:
# faço a definição das variáveis dummy

from pyspark.sql import functions as f

# 1) Coluna de Internet 
dados\
  .groupBy('Id')\
  .pivot('Internet')\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .drop('Nao')\
  .show()

# aqueles que não tiverem serviço de internet ficam com zero nas duas colunas, simples assim.

+----+---+-----------+
|  Id|DSL|FibraOptica|
+----+---+-----------+
|7982|  1|          0|
|9465|  0|          1|
|2122|  1|          0|
|3997|  1|          0|
|6654|  0|          1|
|7880|  0|          1|
|4519|  0|          1|
|6466|  0|          1|
| 496|  1|          0|
|7833|  0|          1|
|1591|  0|          0|
|2866|  0|          1|
|8592|  0|          1|
|1829|  0|          1|
| 463|  0|          1|
|4900|  0|          1|
|4818|  0|          1|
|7554|  1|          0|
|1342|  0|          0|
|5300|  0|          1|
+----+---+-----------+
only showing top 20 rows



In [ ]:
auxiliar_Internet = dados\
  .groupBy('Id')\
  .pivot('Internet')\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .drop('Nao')

In [ ]:
auxiliar_Internet.show()

+----+---+-----------+
|  Id|DSL|FibraOptica|
+----+---+-----------+
|7982|  1|          0|
|9465|  0|          1|
|2122|  1|          0|
|3997|  1|          0|
|6654|  0|          1|
|7880|  0|          1|
|4519|  0|          1|
|6466|  0|          1|
| 496|  1|          0|
|7833|  0|          1|
|1591|  0|          0|
|2866|  0|          1|
|8592|  0|          1|
|1829|  0|          1|
| 463|  0|          1|
|4900|  0|          1|
|4818|  0|          1|
|7554|  1|          0|
|1342|  0|          0|
|5300|  0|          1|
+----+---+-----------+
only showing top 20 rows



In [ ]:
# 2) Coluna TipoContrato
dados\
  .groupBy('Id')\
  .pivot('TipoContrato')\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

+----+--------+-----------+-----+
|  Id|DoisAnos|Mensalmente|UmAno|
+----+--------+-----------+-----+
|7993|       0|          1|    0|
|8592|       0|          1|    0|
|4519|       0|          0|    1|
|1088|       0|          1|    0|
|1238|       0|          1|    0|
|1342|       1|          0|    0|
|4935|       0|          0|    1|
| 471|       0|          1|    0|
|5518|       0|          1|    0|
| 463|       0|          1|    0|
|3794|       0|          1|    0|
|9465|       0|          1|    0|
|7240|       0|          1|    0|
|9852|       0|          1|    0|
|1959|       0|          1|    0|
|7754|       0|          1|    0|
|5156|       0|          0|    1|
|6658|       0|          1|    0|
|6397|       0|          0|    1|
|1829|       0|          1|    0|
+----+--------+-----------+-----+
only showing top 20 rows



In [ ]:
auxiliar_TipoContrato = dados\
  .groupBy('Id')\
  .pivot('TipoContrato')\
  .agg(f.lit(1))\
  .na\
  .fill(0)

In [ ]:
# 3) Coluna MetodoPagamento
dados\
  .groupBy('Id')\
  .pivot('MetodoPagamento')\
  .agg(f.lit(1))\
  .na\
  .fill(0)\
  .show()

+-----+------+----------------+-------------+-------------+
|   Id|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+-----+------+----------------+-------------+-------------+
| 3997|     0|               0|            1|            0|
| 7554|     0|               1|            0|            0|
| 6336|     0|               1|            0|            0|
| 6357|     0|               1|            0|            0|
| 9427|     0|               0|            1|            0|
| 2659|     0|               0|            1|            0|
|  471|     0|               1|            0|            0|
| 4935|     0|               0|            1|            0|
| 4818|     0|               0|            1|            0|
| 1342|     1|               0|            0|            0|
| 1959|     0|               1|            0|            0|
| 9376|     0|               0|            1|            0|
| 2366|     0|               1|            0|            0|
| 1580|     0|               0|         

In [ ]:
auxiliar_MetodoPagamento = dados\
  .groupBy('Id')\
  .pivot('MetodoPagamento')\
  .agg(f.lit(1))\
  .na\
  .fill(0)

In [ ]:
# Agora faço um join desses dataframes auxiliares, incluindo as colunas no dataframe inicial

dados = dados.join(other= auxiliar_Internet, on='Id', how='inner')
dados = dados.join(other= auxiliar_TipoContrato, on='Id', how='inner')
dados = dados.join(other= auxiliar_MetodoPagamento, on='Id', how='inner')

dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+---+-----------+--------+-----------+-----+------+----------------+-------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|DSL|FibraOptica|DoisAnos|Mensalmente|UmAno|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+--

In [ ]:
# agora faço um loop de verificação, alterando tudo que é sim/não para 0/1

from pyspark.sql.types import IntegerType

colunas_binarias = ['Churn', 'Conjuge', 'Dependentes', 'TelefoneFixo', 'MaisDeUmaLinhaTelefonica', 'SegurancaOnline', 'BackupOnline', 'SeguroDispositivo', 'SuporteTecnico', 'TVaCabo', 'StreamingFilmes', 'ContaCorreio']
for col in colunas_binarias:
  dados = dados.withColumn( col, f.regexp_replace(dados[col], 'Sim', '1') )
  dados = dados.withColumn( col, f.regexp_replace(dados[col], 'Nao', '0'))
  dados = dados.withColumn( col, f.regexp_replace(dados[col], 'SemServicoInternet', '0')) # isso aqui estou assumindo que seja assim, mas iria depender da regra de negócio 
  dados = dados.withColumn( col, f.regexp_replace(dados[col], 'SemServicoTelefonico', '0')) # isso aqui estou assumindo que seja assim, mas iria depender da regra de negócio
  
  dados = dados.withColumn(col, dados[col].cast(IntegerType()))

dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+---+-----------+--------+-----------+-----+------+----------------+-------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|DSL|FibraOptica|DoisAnos|Mensalmente|UmAno|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+----------------+-------------+---+-----------+--------+-----------+-----+------+-----

## Transformando os dados (pt2)

- Essa seção se dedica à apresentação dos códigos passados durante as aulas do curso

- Redefino o dataset apenas para mostrar uma outra maneira (talvez mais inteligente e eficiente) de fazer todas as alterações anteriores

In [ ]:
dados = spark.read.csv(
    path = '/content/my-drive/MyDrive/Colab_Notebooks/Alura-CursoSparkModelosClassificacao/dados_clientes.csv',
    sep = ',',
    header = True,
    inferSchema = True

)

dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [ ]:
colunasBinarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [ ]:
todasColunas = [f.when( f.col(c) == 'Sim', 1 ).otherwise(0).alias(c) for c in colunasBinarias ]
todasColunas

[Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorreio = Sim) THEN 1 ELSE 0 END AS ContaCorreio'>]

In [ ]:
# Na lista anterior incluímos todas as listas que receberiam um tratamento de conversão de strings (sim/não -> 1/0)
# A ideia, entretanto, é colocar todas as colunas do conjunto inicial de dados. Portanto, ainda faltam as colunas que, 
# *por enquanto*, ainda não receberam nenhum tratamento.

for column in reversed(dados.columns):
  if column not in colunasBinarias:
    todasColunas.insert(0,column)

todasColunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [ ]:
dados.select(todasColunas).show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

In [ ]:
dataset = dados.select(todasColunas)

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



- Criação de variáveis dummy

In [ ]:
dataset.select(['Internet', 'TipoContrato', 'MetodoPagamento']).show()

# obs.: então o jeito certo de passar colunas no select do pyspark é como lista 
# (não está errado se não passar como lista, mas acho que assim fica mais claro)

+-----------+------------+----------------+
|   Internet|TipoContrato| MetodoPagamento|
+-----------+------------+----------------+
|        DSL| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|          Boleto|
|        DSL| Mensalmente|          Boleto|
|        DSL|       UmAno|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica| Mensalmente|   CartaoCredito|
|        DSL| Mensalmente|          Boleto|
|FibraOptica| Mensalmente|BoletoEletronico|
|        DSL|       UmAno|   DebitoEmConta|
|        DSL| Mensalmente|          Boleto|
|        Nao|    DoisAnos|   CartaoCredito|
|FibraOptica|       UmAno|   CartaoCredito|
|FibraOptica| Mensalmente|   DebitoEmConta|
|FibraOptica| Mensalmente|BoletoEletronico|
|FibraOptica|    DoisAnos|   CartaoCredito|
|        Nao|       UmAno|          Boleto|
|FibraOptica|    DoisAnos|   DebitoEmConta|
|        DSL| Mensalmente|   CartaoCredito|
|FibraOptica| Mensalmente|Boleto

In [ ]:
# Para criar as dummy variables associadas a cada uma das colunas acima podemos utilizar a cláusula pivot

auxiliar_Internet = dataset\
                      .groupBy('Id')\
                      .pivot('Internet')\
                      .agg(f.lit(1))\
                      .na\
                      .fill(0)

auxiliar_Internet.show()

+----+---+-----------+---+
|  Id|DSL|FibraOptica|Nao|
+----+---+-----------+---+
|7982|  1|          0|  0|
|9465|  0|          1|  0|
|2122|  1|          0|  0|
|3997|  1|          0|  0|
|6654|  0|          1|  0|
|7880|  0|          1|  0|
|4519|  0|          1|  0|
|6466|  0|          1|  0|
| 496|  1|          0|  0|
|7833|  0|          1|  0|
|1591|  0|          0|  1|
|2866|  0|          1|  0|
|8592|  0|          1|  0|
|1829|  0|          1|  0|
| 463|  0|          1|  0|
|4900|  0|          1|  0|
|4818|  0|          1|  0|
|7554|  1|          0|  0|
|1342|  0|          0|  1|
|5300|  0|          1|  0|
+----+---+-----------+---+
only showing top 20 rows



In [ ]:
auxiliar_TipoContrato = dataset\
                          .groupBy('Id')\
                          .pivot('TIpoContrato')\
                          .agg(f.lit(1))\
                          .na\
                          .fill(0)
  
auxiliar_TipoContrato.show()

+----+--------+-----------+-----+
|  Id|DoisAnos|Mensalmente|UmAno|
+----+--------+-----------+-----+
|7993|       0|          1|    0|
|8592|       0|          1|    0|
|4519|       0|          0|    1|
|1088|       0|          1|    0|
|1238|       0|          1|    0|
|1342|       1|          0|    0|
|4935|       0|          0|    1|
| 471|       0|          1|    0|
|5518|       0|          1|    0|
| 463|       0|          1|    0|
|3794|       0|          1|    0|
|9465|       0|          1|    0|
|7240|       0|          1|    0|
|9852|       0|          1|    0|
|1959|       0|          1|    0|
|7754|       0|          1|    0|
|5156|       0|          0|    1|
|6658|       0|          1|    0|
|6397|       0|          0|    1|
|1829|       0|          1|    0|
+----+--------+-----------+-----+
only showing top 20 rows



In [ ]:
auxiliar_MetodoPagamento = dataset\
                              .groupBy('Id')\
                              .pivot('MetodoPagamento')\
                              .agg(f.lit(1))\
                              .na\
                              .fill(0)

auxiliar_MetodoPagamento.show()                     

+-----+------+----------------+-------------+-------------+
|   Id|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+-----+------+----------------+-------------+-------------+
| 3997|     0|               0|            1|            0|
| 7554|     0|               1|            0|            0|
| 6336|     0|               1|            0|            0|
| 6357|     0|               1|            0|            0|
| 9427|     0|               0|            1|            0|
| 2659|     0|               0|            1|            0|
|  471|     0|               1|            0|            0|
| 4935|     0|               0|            1|            0|
| 4818|     0|               0|            1|            0|
| 1342|     1|               0|            0|            0|
| 1959|     0|               1|            0|            0|
| 9376|     0|               0|            1|            0|
| 2366|     0|               1|            0|            0|
| 1580|     0|               0|         

In [ ]:
# Agora eu faço a união das tabelas anteriores com o nosso dataset inicial, para incluir essas colunas
# dummy no conjunto de trabalho.

# Aproveitando que estamos fazendo as manipulações finais no dataset em questão, já faço uma alteração dos nomes
# das colunas novas geradas (para saber de onde, no dataset original, elas vieram) e excluo as variáveis 
# categóricas originais correspondentes, que não serão mais úteis.

dataset\
  .join(other=auxiliar_Internet, on='Id', how='inner')\
  .join(other=auxiliar_TipoContrato, on='Id', how='inner')\
  .join(other=auxiliar_MetodoPagamento, on='Id', how='inner')\
  .withColumnRenamed('DSL','Internet_DSL')\
  .withColumnRenamed('FibraOptica','Internet_FibraOptica')\
  .withColumnRenamed('Nao','Internet_Nao')\
  .withColumnRenamed('DoisAnos','TipoContrato_DoisAnos')\
  .withColumnRenamed('Mensalmente','TipoContrato_Mensalmente')\
  .withColumnRenamed('UmAno','TipoContrato_UmAno')\
  .withColumnRenamed('Boleto','MetodoPagamento_Boleto')\
  .withColumnRenamed('BoletoEletronico','MetodoPagamento_BoletoEletronico')\
  .withColumnRenamed('CartaoCredito','MetodoPagamento_CartaoCredito')\
  .withColumnRenamed('DebitoEmConta','MetodoPagamento_DebitoEmConta')\
  .drop('Internet', 'TipoContrato', 'MetodoPagamento')\
  .show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+---------------------+------------------------+------------------+----------------------+--------------------------------+-----------------------------+-----------------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
# Faço uma atualização no dataset de trabalho, redefinindo-o com base nas modificações anteriores

dataset = dataset\
  .join(other=auxiliar_Internet, on='Id', how='inner')\
  .join(other=auxiliar_TipoContrato, on='Id', how='inner')\
  .join(other=auxiliar_MetodoPagamento, on='Id', how='inner')\
  .withColumnRenamed('DSL','Internet_DSL')\
  .withColumnRenamed('FibraOptica','Internet_FibraOptica')\
  .withColumnRenamed('Nao','Internet_Nao')\
  .withColumnRenamed('DoisAnos','TipoContrato_DoisAnos')\
  .withColumnRenamed('Mensalmente','TipoContrato_Mensalmente')\
  .withColumnRenamed('UmAno','TipoContrato_UmAno')\
  .withColumnRenamed('Boleto','MetodoPagamento_Boleto')\
  .withColumnRenamed('BoletoEletronico','MetodoPagamento_BoletoEletronico')\
  .withColumnRenamed('CartaoCredito','MetodoPagamento_CartaoCredito')\
  .withColumnRenamed('DebitoEmConta','MetodoPagamento_DebitoEmConta')\
  .drop('Internet', 'TipoContrato', 'MetodoPagamento')

In [ ]:
dataset.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+---------------------+------------------------+------------------+----------------------+--------------------------------+-----------------------------+-----------------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_DoisAnos: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoCo

## Regressão Logística

- Preparando os dados

In [ ]:
dataset.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+---------------------+------------------------+------------------+----------------------+--------------------------------+-----------------------------+-----------------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

In [ ]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_DoisAnos: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoCo

In [ ]:
# Efetuo uma vetorização do meu dataframe incluindo todas as variáveis (features) importantes

from pyspark.ml.feature import VectorAssembler

# defino as features que quero incluir no modelo
X = [
'Mais65anos',
'MesesDeContrato',
'MesesCobrados',
'Conjuge',
'Dependentes',
'MaisDeUmaLinhaTelefonica',
'TelefoneFixo',
'SegurancaOnline',
'BackupOnline',
'SeguroDispositivo',
'SuporteTecnico',
'TVaCabo',
'StreamingFilmes',
'ContaCorreio',
'Internet_DSL',
'Internet_FibraOptica',
'Internet_Nao',
'TipoContrato_DoisAnos',
'TipoContrato_Mensalmente',
'TipoContrato_UmAno',
'MetodoPagamento_Boleto',
'MetodoPagamento_BoletoEletronico',
'MetodoPagamento_CartaoCredito',
'MetodoPagamento_DebitoEmConta'
] # No curso a coluna 'telefone fixo' foi incluída, mas eu não acho que é relevante para o estudo...

features_vetorizadas = VectorAssembler(inputCols=X, outputCol='features') # definição das features vetorizadas
dataset = dataset.withColumnRenamed('Churn','label') # aproveito para mudar o nome da coluna que quero prever com o modelo

# defino um novo dataset contendo somente as informações de interesse para a execução do modelo: features / label
# isso é feito incluindo a coluna 'features' por meio do objeto features_vetorizadas + .transform(dataset)
# e selecionando somente as duas colunas de interesse
dataset_vec =  features_vetorizadas.transform(dataset).select('features', 'label')
dataset_vec.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(24,[1,2,11,12,13...|    1|
|(24,[1,2,3,5,6,8,...|    1|
|(24,[1,2,5,6,10,1...|    0|
|(24,[1,2,3,6,8,12...|    0|
|(24,[1,2,3,5,6,11...|    1|
|(24,[1,2,5,6,12,1...|    1|
|(24,[1,2,3,5,6,8,...|    0|
|(24,[1,2,5,6,15,1...|    0|
|(24,[1,2,3,6,7,8,...|    0|
|(24,[1,2,3,6,12,1...|    1|
|(24,[1,2,6,16,18,...|    0|
|(24,[1,2,6,8,12,1...|    0|
|(24,[1,2,3,5,6,11...|    1|
|(24,[1,2,5,6,13,1...|    0|
|(24,[1,2,5,6,8,11...|    1|
|(24,[0,1,2,3,5,6,...|    1|
|(24,[0,1,2,3,6,7,...|    0|
|(24,[1,2,6,8,14,1...|    1|
|(24,[1,2,6,16,17,...|    0|
|(24,[1,2,3,4,6,11...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
'''

Uma outra maneira de definir objeto lista X seria a seguinte:

X = dataset.columns
X.remove('Churn')
X.remove('Id')

(já pegaria tudo de uma vez, e aí a gente só exclui as colunas / variáveis que não servem para o modelo:
o Id dos clientes e o Churn, que é a variável que queremos prever)

'''

"\n\nUma outra maneira de definir objeto lista X seria a seguinte:\n\nX = dataset.columns\nX.remove('Churn')\nX.remove('Id')\n\n(já pegaria tudo de uma vez, e aí a gente só exclui as colunas / variáveis que não servem para o modelo:\no Id dos clientes e o Churn, que é a variável que queremos prever)\n\n"

In [ ]:
dataset_vec.show(20, truncate=False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,18,21],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,18,21],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,19,20],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,6,8,12,13,14,17,22],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,18,21],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,18,21],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

- Observações sobre a coluna de features gerada:
  - O primeiro campo contém a quantidade de features definida (no nosso caso, 24, que fica como default em todas as linhas).
  - O segundo campo (lista) contém a informações de quais colunas (features) têm valores diferentes de 0 para aquela linha específica. Como o Spark busca eficiência máxima no tratamento de grandes volumes de dados, valores nulos nem são armazenados. (matriz sparse)
  - Por fim, o terceiro campo (lista) contém os valores não nulos de cada uma das colunas (features) indicada no campo anterior.

In [ ]:
# por fim, antes de seguir para a construção do modelo e obter as previsões, defino os datasets de treino e teste

SEED = 101
treino, teste = dataset_vec.randomSplit(weights=[0.7,0.3], seed = SEED)

- Construindo o modelo

In [ ]:
# importo a biblioteca 
from pyspark.ml.classification import LogisticRegression

In [ ]:
# defino o objeto regressor
lr = LogisticRegression()

In [ ]:
# defino o modelo, que consiste num ajuste dos dados de treino (considerando toda a escolha do método de regressão,
# features definidas e preparação do dataset)
modelo_lr = lr.fit(treino)

In [ ]:
# obtenho as previsões do modelo para o conjunto de teste

previsoes_lr_teste = modelo_lr.transform(teste)
previsoes_lr_teste.show(5,truncate=False)

+------------------------------------------------------------------------------------------------------+-----+------------------------------------------+----------------------------------------+----------+
|features                                                                                              |label|rawPrediction                             |probability                             |prediction|
+------------------------------------------------------------------------------------------------------+-----+------------------------------------------+----------------------------------------+----------+
|(24,[0,1,2,3,4,5,6,7,9,10,11,14,19,21],[1.0,55.0,76.25,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])  |0    |[3.005148326895531,-3.005148326895531]    |[0.9528061698620683,0.04719383013793166]|0.0       |
|(24,[0,1,2,3,4,5,6,7,13,15,18,21],[1.0,24.0,79.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])               |0    |[-0.14296194505890192,0.14296194505890192]|[0.4643202619359225,0.5

## Métricas de qualidade (Logistic Regression)

In [ ]:
# definindo o resumo do ajuste
resumo_lr_treino = modelo_lr.summary

In [ ]:
print("Acurácia: %f" % resumo_lr_treino.accuracy)
print("Precisão: %f" % resumo_lr_treino.precisionByLabel[1])
print("Recall: %f" % resumo_lr_treino.recallByLabel[1])
print("F1: %f" % resumo_lr_treino.fMeasureByLabel()[1])

Acurácia: 0.783791
Precisão: 0.768162
Recall: 0.812934
F1: 0.789914


In [ ]:
# Para verificar a qualidade do modelo com respeito aos dados de teste, construíremos um objeto
# chamado "matriz de confusão"

# Para construir essa matriz, é necessário definir as seguintes quantidades
  # - True Positive (TP): registros positivos (1) que foram previstos positivos (1)
  # - False Positive (FP): registros negativos (0) que foram previstos positivos (1)
  # - False Negative (FN): registros positivos (1) que foram previstos negativos (0)
  # - True Negative (TN): registros negativos (0) que foram previstos negativos (0)

In [ ]:
previsoes_lr_teste.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       1.0|
|    1|       0.0|
|    0|       1.0|
|    0|       1.0|
+-----+----------+
only showing top 5 rows



In [ ]:
tp = previsoes_lr_teste.filter( (f.col('label') == 1) & (f.col('prediction') == 1) ).count()
fp = previsoes_lr_teste.filter( (f.col('label') == 0) & (f.col('prediction') == 1) ).count()
fn = previsoes_lr_teste.filter( (previsoes_lr_teste['label'] == 1) & (previsoes_lr_teste['prediction'] == 0) ).count()
tn = previsoes_lr_teste.where( (f.col('label') == 0) & (f.col('prediction') == 0) ).count()

print(tp,tn, fp, fn)

1260 1175 396 311


In [ ]:
def apresenta_matriz_confusao(df_transform_modelo, normalize=False, percentage=True):
  tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
  tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
  fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
  fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

  valorP = 1
  valorN = 1

# isso é regra de normalização que não foi discutida (definido assim)
  if normalize == True:
    valorP = tp + fn
    valorN = fp + tn

# esse trecho apenas transforma os valores de normalização para porcentagem
  if percentage == True and normalize == True:
    valorP = valorP / 100
    valorN = valorN / 100

  print(' '*20, 'Previsto')
  print(' '*15, 'Churn', ' '*5 ,'Não-Churn')
  print(' '*4, 'Churn', ' '*6, int(tp/valorP), ' '*7, int(fn/valorP))
  print('Real')
  print(' '*4, 'Não-Churn', ' '*2, int(fp/valorN), ' '*7, int(tn/valorN))

In [ ]:
apresenta_matriz_confusao(previsoes_lr_teste)

                     Previsto
                Churn       Não-Churn
     Churn        1260         311
Real
     Não-Churn    396         1175


In [ ]:
#from utils import calcula_mostra_metricas

In [ ]:
#print(calcula_mostra_metricas(modelo_lr, previsoes_lr_teste))

## Árvore de decisão (adaptada para classificação)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
# definição do objeto classificador
dtc = DecisionTreeClassifier(seed = SEED) 

# O dtc utiliza um método randômico para avaliação das features, então
# precisamos aplicar uma seed para garantir que obteremos sempre o mesmo resultado.

In [ ]:
modelo_dtc = dtc.fit(treino)

In [ ]:
previsoes_dtc_teste = modelo_dtc.transform(teste)

In [ ]:
previsoes_dtc_teste.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2059.0,329.0]|[0.86222780569514...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [110.0,171.0]|[0.39145907473309...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [244.0,211.0]|[0.53626373626373...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[464.0,2084.0]|[0.18210361067503...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [244.0,211.0]|[0.53626373626373...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [43.0,132.0]|[0.24571428571428...|       1.0|
|(24,[0,1,2,3,4,5,...|    1| [244.0,211.0]|[0.53626373626373...|       0.0|
|(24,[0,1,2,3,4,5,...|    0| [244.0,211.0]|[0.53626373626373...|       0.0|
|(24,[0,1,2,3,4,6,...|    0|[464.0,2084.0]|[0.18210361067503...|       1.0|
|(24,[0,1,2,3,4,6,...|    0|[2059.0,329.0]|[0.86222780569514...|       0.0|
|(24,[0,1,2,

## Métricas de qualidade (Decision Tree Classifier)

In [ ]:
# Aqui é necessário importar uma biblioteca específica para avaliar as métricas
# do modelo atual (que constitui uma previsão de classificação)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# dica: digitando ctlr+espaço -> o colab tenta preencher automaticamente pra gente

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
print(evaluator.evaluate(previsoes_dtc_teste , {evaluator.metricName:'accuracy'}))

0.7730744748567792


In [ ]:
print('Decision Tree Classifier')
print("="*40)
print("Dados de Teste (total de {} dados)".format(teste.count()))
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_dtc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier
Dados de Teste (total de 3142 dados)
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1259         312
Real
     Não-Churn    401         1170
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.773074
Precisão: 0.758434
Recall: 0.801400
F1: 0.779325


## Random Forest (adaptado para classificação)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
# definição do objeto classificador
rfc = RandomForestClassifier(seed = SEED, numTrees = 3)

In [ ]:
modelo_rfc = rfc.fit(treino)

In [ ]:
previsoes_rfc_treino = modelo_rfc.transform(treino)

In [ ]:
previsoes_rfc_treino.show(4, truncate=False)

+----------------------------------------------------------------------------------------------------+-----+---------------------------------------+----------------------------------------+----------+
|features                                                                                            |label|rawPrediction                          |probability                             |prediction|
+----------------------------------------------------------------------------------------------------+-----+---------------------------------------+----------------------------------------+----------+
|(24,[0,1,2,3,4,5,6,7,8,10,13,15,19,23],[1.0,58.0,89.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0    |[2.2648987441219397,0.7351012558780605]|[0.7549662480406466,0.2450337519593535] |0.0       |
|(24,[0,1,2,3,4,5,6,7,8,11,14,19,21],[1.0,71.0,69.2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])        |0    |[2.2200442640915985,0.7799557359084014]|[0.7400147546971995,0.2599852453028005] |0.0    

## Métricas de qualidade (Random Forest Classifier)

In [ ]:
# antes de seguir para a avaliação das métricas, defino o dataset de previsões do conjunto teste
previsoes_rfc_teste = modelo_rfc.transform(teste)

In [ ]:
print('Random Forest Classifier')
print("="*40)
print("Dados de Treino (total de {} dados)".format(treino.count()))
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_rfc_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste (total de {} dados)".format(teste.count()))
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_rfc_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier
Dados de Treino (total de 7206 dados)
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2635         968
Real
     Não-Churn    727         2876
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.764779
Precisão: 0.783760
Recall: 0.731335
F1: 0.756640

Dados de Teste (total de 3142 dados)
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1143         428
Real
     Não-Churn    329         1242
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.759071
Precisão: 0.776495
Recall: 0.727562
F1: 0.751232


In [ ]:
previsoes_lr_treino = modelo_lr.transform(treino)

print('Random Forest Classifier')
print("="*40)
print("Dados de Treino")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_lr_treino, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_lr_treino, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_lr_treino, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_lr_treino, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_lr_treino, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))
print("")
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_lr_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_lr_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier
Dados de Treino
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        2929         674
Real
     Não-Churn    884         2719
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.783791
Precisão: 0.768162
Recall: 0.812934
F1: 0.789914

Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1260         311
Real
     Não-Churn    396         1175
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.774984
Precisão: 0.760870
Recall: 0.802037
F1: 0.780911


In [ ]:
def compara_metricas_modelos(lista_previsoes):

  # ‘s’ será minha string de retorno
  # ela vai coletar e montar minha matriz de confusão
  # e também os valores de acurácia, precisão, recall e F1-score

  # lista_previsoes deve ser um dicionário de dados
  s = '\n'

  for modelo, df_transform_modelo in lista_previsoes.items():

    s += '-' * 50 + '\n' #linha de separação
    s += modelo + '\n'

    # os passos para montagem da matriz de confusão são os mesmos da aula
    tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
    tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
    fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
    fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

    # construção da minha string da matriz de confusão  
    s += ' '*20 + 'Previsto\n'
    s += ' '*15 +  'Churn' + ' '*5 + 'Não-Churn\n'
    s += ' '*4 + 'Churn' + ' '*6 +  str(int(tp)) + ' '*7 + str(int(fn)) + '\n'
    s += 'Real\n'
    s += ' '*4 + 'Não-Churn' + ' '*2 + str(int(fp)) +  ' '*7 + str(int(tn))  + '\n'
    s += '\n'

    # adiciono os valores de cada métrica a minha string de retorno com MulticlassClassificationEvaluator
    evaluator = MulticlassClassificationEvaluator()

    s += f'Acurácia: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "accuracy"})}\n'
    s += f'Precisão: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}\n'
    s += f'Recall: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}\n'
    s += f'F1: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}\n'

  return s

In [ ]:
dicionario_modelos = {'Logistic Regression': previsoes_lr_teste,
                                'DecisionTreeClassifier': previsoes_dtc_teste,
                                'RandomForestClassifier': previsoes_rfc_teste}

In [ ]:
print(compara_metricas_modelos(dicionario_modelos))


--------------------------------------------------
Logistic Regression
                    Previsto
               Churn     Não-Churn
    Churn      1260       311
Real
    Não-Churn  396       1175

Acurácia: 0.7749840865690643
Precisão: 0.7608695652173914
Recall: 0.8020369191597708
F1: 0.7809110629067245
--------------------------------------------------
DecisionTreeClassifier
                    Previsto
               Churn     Não-Churn
    Churn      1259       312
Real
    Não-Churn  401       1170

Acurácia: 0.7730744748567792
Precisão: 0.7584337349397591
Recall: 0.8014003819223424
F1: 0.7793252862890747
--------------------------------------------------
RandomForestClassifier
                    Previsto
               Churn     Não-Churn
    Churn      1143       428
Real
    Não-Churn  329       1242

Acurácia: 0.7590706556333545
Precisão: 0.7764945652173914
Recall: 0.7275620623806492
F1: 0.7512323365100232



- Relembrando a regra de negócio inicial: queremos construir um modelo de previsão para identificar quais clientes podem cancelar (dar churn) no serviço fornecido pela empresa. A ideia é passar essas informações para a equipe de marketing que, posteriormente, entrará em contato e fornecerá promoções/descontos a fim de evitar o cancelamento.

- Em vista disso, queremos, claro, um modelo com boa acurácia, ou seja, com um bom grau de acertos levando em conta as bases de dados de treino/teste fornecidos. Contudo, precisamos, acima de tudo, que o modelo tenha a melhor precisão possível (i.e., menor quantidade de falsos positivos possível). Por quê? Pelo seguinte motivo:
  - A ideia é que os clientes com potencial de churn previstos pelo modelo contruído recebam promoções / descontos da equipe de marketing, a fim de evitar o cancelamento dos serviços. Se o modelo estiver prevendo muitos clientes com potencial de churn, mas que não pretendiam originalmente cancelar o serviço, a empresa sairá no prejuízo. Clientes que continuariam assinando o serviço de qualquer maneira receberão promoções / descontos "desnecessários"

  - A métrica recall também tem extrema importância. Se o modelo prevê erroneamente clientes que não tem potencial de cancelar, mas que pretendem cancelar, isso significa que não estaremos dando a devida atenção a esses casos e, efetivamente, a empresa perderá clientes (i.e., lucro).


- Resumo da ópera: dentro dessa regra de negócio, é melhor ter uma quantidade menor de falsos negativos (ou seja, é melhor termos em nossa vista um número maior de clientes que pretendem cancelar) do que falsos positivos (clientes que não pretendem cancelar, mas que receberão um descono mesmo assim). Dessa forma garantimos que a empresa não perderá um número expressivo de compradores.

## Implementando o Cross Validation (tunning)

- Decision Tree com Cross Validation

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
dtc = DecisionTreeClassifier(seed=SEED)

In [ ]:
grid = ParamGridBuilder()\
        .addGrid(dtc.maxDepth, [2,5,10])\
        .addGrid(dtc.maxBins, [10, 32, 45])\
        .build()

In [ ]:
evaluator = MulticlassClassificationEvaluator()

In [ ]:
dtc_cv = CrossValidator(
    estimator = dtc,
    estimatorParamMaps = grid,
    seed = SEED,
    evaluator = evaluator,
    numFolds = 3
)

In [ ]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [ ]:
previsoes_dtc_cv_teste = modelo_dtc_cv.transform(teste)

In [ ]:
previsoes_dtc_cv_teste.show()

+--------------------+-----+-------------+--------------------+----------+
|            features|label|rawPrediction|         probability|prediction|
+--------------------+-----+-------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|   [13.0,1.0]|[0.92857142857142...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [45.0,76.0]|[0.37190082644628...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|  [49.0,83.0]|[0.37121212121212...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [35.0,328.0]|[0.09641873278236...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|  [50.0,39.0]|[0.56179775280898...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [4.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|  [49.0,83.0]|[0.37121212121212...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|  [49.0,83.0]|[0.37121212121212...|       1.0|
|(24,[0,1,2,3,4,6,...|    0|   [24.0,2.0]|[0.92307692307692...|       0.0|
|(24,[0,1,2,3,4,6,...|    0|   [56.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,6,...|   

In [ ]:
print('Decision Tree Classifier - Tuning')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_dtc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_dtc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Decision Tree Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1281         290
Real
     Não-Churn    377         1194
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.787715
Precisão: 0.772618
Recall: 0.815404
F1: 0.793434


- Random Forest com Cross Validation

In [ ]:
rfc = RandomForestClassifier(seed=SEED)

In [ ]:
grid = ParamGridBuilder()\
          .addGrid(rfc.maxDepth, [2,5,10])\
          .addGrid(rfc.maxBins, [10, 32, 45])\
          .addGrid(rfc.numTrees, [10, 20 ,30])\
          .build()

In [ ]:
rfc_cv = CrossValidator(
    estimator = rfc,
    estimatorParamMaps = grid,
    evaluator = evaluator,
    numFolds = 3,
    seed = SEED
)

In [ ]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [ ]:
previsoes_rfc_cv_teste = modelo_rfc_cv.transform(teste)

In [ ]:
previsoes_rfc_cv_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[19.4071195495544...|[0.97035597747772...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[8.78014608472799...|[0.43900730423639...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[16.3010960418000...|[0.81505480209000...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.26821004267078...|[0.36341050213353...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[12.7077505584323...|[0.63538752792161...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[7.62451230138597...|[0.38122561506929...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[8.84173637924702...|[0.44208681896235...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[10.9340990203505...|[0.54670495101752...|       0.0|
|(24,[0,1,2,3,4,6,...|    0|[14.7517228306702...|[0.73758614153351...|       0.0|
|(24,[0,1,2,3,4,

In [ ]:
print('Random Forest Classifier - Tuning')
print("="*40)
print("Dados de Teste")
print("="*40)
print("Matriz de Confusão")
print("-"*40)
apresenta_matriz_confusao(previsoes_rfc_cv_teste, normalize=False)
print("-"*40)
print("Métricas")
print("-"*40)
print("Acurácia: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "accuracy"}))
print("Precisão: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1}))
print("Recall: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1}))
print("F1: %f" % evaluator.evaluate(previsoes_rfc_cv_teste, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1}))

Random Forest Classifier - Tuning
Dados de Teste
Matriz de Confusão
----------------------------------------
                     Previsto
                Churn       Não-Churn
     Churn        1323         248
Real
     Não-Churn    327         1244
----------------------------------------
Métricas
----------------------------------------
Acurácia: 0.816996
Precisão: 0.801818
Recall: 0.842139
F1: 0.821484


In [ ]:
# ao avaliar as métricas e comparar com os resultados anteriores, concluímos
# que o rfc com cross validator é realmente o melhor modelo para implementarmos
# no projeto

melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [ ]:
print(melhor_modelo_rfc_cv.getMaxDepth())
print(melhor_modelo_rfc_cv.getMaxBins())
print(melhor_modelo_rfc_cv.getNumTrees)

10
45
20


In [ ]:
rfc_tunned = RandomForestClassifier(seed=SEED, maxDepth=10, maxBins=45, numTrees=10)

In [ ]:
# aqui agora estamos lidando com o modelo final, aquele que será utilizado
# para prever resultados quando novos dados chegarem

# a nossa validação já foi feita, então aqui nós vamos passar o dataset completo
# de dados (para obter as melhores informações / previsões possíveis)
modelo_rfc_tunned = rfc_tunned.fit(dataset_vec)

- Como utilizar o modelo?

In [ ]:
novo_cliente = [{
    'Mais65anos': 0,
    'MesesDeContrato': 1,
    'MesesCobrados': 45.30540797610398,
    'Conjuge': 0,
    'Dependentes': 0,
    'TelefoneFixo': 0,
    'MaisDeUmaLinhaTelefonica': 0,
    'SegurancaOnline': 0,
    'BackupOnline': 0,
    'SeguroDispositivo': 0,
    'SuporteTecnico': 0,
    'TVaCabo': 1,
    'StreamingFilmes': 1,
    'ContaCorreio': 1,
    'Internet_DSL': 1,
    'Internet_FibraOptica': 0,
    'Internet_Nao': 0,
    'TipoContrato_Mensalmente': 1,
    'TipoContrato_UmAno': 0,
    'TipoContrato_DoisAnos': 0,
    'MetodoPagamento_DebitoEmConta': 0,
    'MetodoPagamento_CartaoCredito': 0,
    'MetodoPagamento_BoletoEletronico': 1,
    'MetodoPagamento_Boleto': 0
}]

In [ ]:
novo_cliente = spark.createDataFrame(novo_cliente)
novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [ ]:
assembler = VectorAssembler(inputCols = X, outputCol = 'features')

In [ ]:
novo_cliente_vetorizado = assembler.transform(novo_cliente).select('features')
novo_cliente_vetorizado.show()

# não temos mais a coluna de label, somente as features de estudo 
# (aqui o modelo já tá construído, e o que nós queremos fazer é aplicá-lo
# e determinar se o cliente vai virar churn ou não)

+--------------------+
|            features|
+--------------------+
|(24,[1,2,11,12,13...|
+--------------------+



In [ ]:
previsao_final = modelo_rfc_tunned.transform(novo_cliente_vetorizado)
previsao_final.show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[1.08463334203136...|[0.10846333420313...|       1.0|
+--------------------+--------------------+--------------------+----------+



## Conclusão do curso

- Tunning do modelo de regressão logística

In [ ]:
# queremos obter um modelo de regressão logística otimizado, então aplico novamente todas os conceitos já vistos
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# definindo o modelo regressor 
lr = LogisticRegression()

In [ ]:
lr?

In [ ]:
# busco quais são os parâmetros relevantes desse estimador e defino
# o grid de trabalho

grid = ParamGridBuilder()\
        .addGrid(lr.maxIter, [100,150,200])\
        .addGrid(lr.aggregationDepth, [2,5,10])\
        .build()

In [ ]:
'''
No exemplo da aula, por exemplo, o instrutor mexeu nos parâmetros elasticNetParam e regParam
'''

In [ ]:
# definição do objeto avaliador (verificação de qualidade do modelo)
evaluator = MulticlassClassificationEvaluator()

In [ ]:
# definição do objeto de tuning
lr_cv = CrossValidator(
    estimator = lr,
    estimatorParamMaps = grid,
    evaluator = evaluator,
    numFolds = 5
)

In [ ]:
# definição do modelo
modelo_lr_cv = lr_cv.fit(treino)

In [ ]:
# obtenção das previsões
previsoes_lr_cv_teste = modelo_lr_cv.transform(teste)

In [ ]:
previsoes_lr_cv_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[3.00514832689553...|[0.95280616986206...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1429619450589...|[0.46432026193592...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.14269412787148...|[0.53561312407406...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[-1.1382630453467...|[0.24263941068065...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.1931900449138...|[0.45185214513604...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[-0.3825712888226...|[0.40550688346460...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[0.02002179187065...|[0.50500528076230...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.13275563393771...|[0.53314025063255...|       0.0|
|(24,[0,1,2,3,4,6,...|    0|[0.71012172932930...|[0.67042805698284...|       0.0|
|(24,[0,1,2,3,4,

In [ ]:
# verificação de métricas
print('Accuracy: {}'.format(evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName:'accuracy'} )))
print('Precision: {}'.format(evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1})))
print('Recall: {}'.format(evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1})))
print('F1 Score: {}'.format(evaluator.evaluate(previsoes_lr_cv_teste, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1})))

# Obs.: o metricLabel = 1 significa que estamos calculando as métricas com respeito à definição associada às saídas (corretas) positivas do modelo.
# Ou seja: Precisao = TP/(TP+FP), Recall = TP/(TP+FN) ...

# Definir o metricLabel = 0 significaria calcular as métricas com respeito às saídas (corretas) negativas do modelo.
# Ou seja: Precisao = TN/(TN+FP), Recall = TN/(TN+FN)

# As duas definições terão valores diferentes, mas uma vez que uma "convenção" seja adotada, as informações obtidas de ambas são totalmente equivalentes
# (basta adotar uma e seguir até o fim)

Accuracy: 0.7749840865690643
Precision: 0.7608695652173914
Recall: 0.8020369191597708
F1 Score: 0.7809110629067245


In [ ]:
# Verificação do melhor modelo
print(modelo_lr_cv.bestModel.getMaxIter())
print(modelo_lr_cv.bestModel.getAggregationDepth())

100
2
